In [2]:
# YYYYMMDD = Datum (YYYY=jaar MM=maand DD=dag); 
# DDVEC    = Vectorgemiddelde windrichting in graden (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil/variabel). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken; 
# FHVEC    = Vectorgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken; 
# FG       = Etmaalgemiddelde windsnelheid (in 0.1 m/s); 
# FHX      = Hoogste uurgemiddelde windsnelheid (in 0.1 m/s); 
# FHXH     = Uurvak waarin FHX is gemeten; 
# FHN      = Laagste uurgemiddelde windsnelheid (in 0.1 m/s); 
# FHNH     = Uurvak waarin FHN is gemeten; 
# FXX      = Hoogste windstoot (in 0.1 m/s); 
# FXXH     = Uurvak waarin FXX is gemeten; 
# TG       = Etmaalgemiddelde temperatuur (in 0.1 graden Celsius); 
# TN       = Minimum temperatuur (in 0.1 graden Celsius); 
# TNH      = Uurvak waarin TN is gemeten; 
# TX       = Maximum temperatuur (in 0.1 graden Celsius); 
# TXH      = Uurvak waarin TX is gemeten; 
# T10N     = Minimum temperatuur op 10 cm hoogte (in 0.1 graden Celsius); 
# T10NH    = 6-uurs tijdvak waarin T10N is gemeten; 6=0-6 UT, 12=6-12 UT, 18=12-18 UT, 24=18-24 UT
# SQ       = Zonneschijnduur (in 0.1 uur) berekend uit de globale straling (-1 voor <0.05 uur); 
# SP       = Percentage van de langst mogelijke zonneschijnduur; 
# Q        = Globale straling (in J/cm2); 
# DR       = Duur van de neerslag (in 0.1 uur); 
# RH       = Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm); 
# RHX      = Hoogste uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm); 
# RHXH     = Uurvak waarin RHX is gemeten; 
# PG       = Etmaalgemiddelde luchtdruk herleid tot zeeniveau (in 0.1 hPa) berekend uit 24 uurwaarden; 
# PX       = Hoogste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa); 
# PXH      = Uurvak waarin PX is gemeten; 
# PN       = Laagste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa); 
# PNH      = Uurvak waarin PN is gemeten; 
# VVN      = Minimum opgetreden zicht; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)
# VVNH     = Uurvak waarin VVN is gemeten; 
# VVX      = Maximum opgetreden zicht; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)
# VVXH     = Uurvak waarin VVX is gemeten; 
# NG       = Etmaalgemiddelde bewolking (bedekkingsgraad van de bovenlucht in achtsten, 9=bovenlucht onzichtbaar); 
# UG       = Etmaalgemiddelde relatieve vochtigheid (in procenten); 
# UX       = Maximale relatieve vochtigheid (in procenten); 
# UXH      = Uurvak waarin UX is gemeten; 
# UN       = Minimale relatieve vochtigheid (in procenten); 
# UNH      = Uurvak waarin UN is gemeten; 
# EV24     = Referentiegewasverdamping (Makkink) (in 0.1 mm);

In [5]:
# http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi

import pandas as pd
import numpy as np

df = pd.read_csv('data/weather.csv', low_memory=False)
df

,STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,...,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV24
0,IJMOND,20010130,,,,,,,,,...,,,,,,,,,,
1,IJMOND,20010131,,,,,,,,,...,,,,,,,,,,
2,IJMOND,20010201,236,13,24,50,8,10,1,90,...,,,,,,,,,,
3,IJMOND,20010202,150,46,58,80,9,10,3,100,...,,,,,,,,,,
4,IJMOND,20010203,106,65,66,90,8,50,1,120,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347476,ARCEN,20191206,215,56,57,70,20,40,4,140,...,,,,,94,97,10,87,4,1
347477,ARCEN,20191207,236,40,42,50,1,30,18,100,...,,,,,86,96,2,79,19,2
347478,ARCEN,20191208,226,58,59,80,7,40,1,170,...,,,,,81,96,10,73,15,2
347479,ARCEN,20191209,245,34,40,60,1,20,15,130,...,,,,,87,97,9,77,3,2
